# Relevamiento de carga de datos sospechosa en las elecciones de Provincia de Córdoba

Este análisis es mi aporte para hacer el trabajo de **chequeo manual de telegramas de las PASO 2017** en Córdoba detectando aquellas resultados estádisticamente raras (no quiere decir que todas las mesas listadas estén incorrectas). 

Para esto realizé un programa que obtiene los datos digitalizados de cada una de las mesas y los estructuré en una base de datos para hacer agregaciones y consultas. 

Por ejemplo, basado en la idea de que los resultados en una mesa son similares en un circuito, se pueden detectar aquellas mesas que tienen resultados muy distintos. 

Las tablas resultantes muestran algunos datos e incluyen el link para ver el telegrama en el sitio oficial para que revisemos entre tod@s. 

Todo el [código fuente está disponible en github](http://github.com/mgaitan/mesas_ba/)

**Gracias por difundir**

- Martín Gaitán [`@tin_nqn_`](https://twitter.com/tin_nqn_)



<div class="alert alert-info" role="alert">Si querés podés <a href="#resultados">ir a los resultados</a>




</div>



### Generación de base de datos

Utilizo el ORM de Django para modelar la base de datos y luego importo los CSVs del [crawler del escrutinio provisorio](https://github.com/mgaitan/mesas_ba/tree/master/mesas_ba), mesa por mesa. 


In [1]:
%load_ext django_orm_magic

In [2]:
%%django_orm
from django.db import models
from django.db.models import Sum

class Seccion(models.Model):
    numero = models.CharField(max_length=10)
    nombre = models.CharField(max_length=100)
    
    def __str__(self):
        return f'{self.numero} - {self.nombre}'

class Circuito(models.Model):
    numero = models.CharField(max_length=10)
    seccion = models.ForeignKey('Seccion')

    def __str__(self):
        return u"Circuito %s (%s)" % (self.numero, self.seccion)
    
class Mesa(models.Model):
    numero = models.CharField(max_length=10)
    circuito = models.ForeignKey('Circuito')
    fake_id = models.CharField(max_length=100, unique=True)
    url = models.CharField(max_length=200)
    
    class Meta:
        unique_together = ('circuito', 'numero')
    
    @property
    def computados(self):
        return self.resultados.aggregate(Sum('votos'))['votos__sum']

    def __str__(self):
        return u"Mesa %s (%s)" % (self.numero, self.circuito)
    
    @property
    def absolute_url(self):
        return f'http://www.resultados.gob.ar/99/resu/content/telegramas/{self.url}'
    

class Partido(models.Model):
    nombre = models.CharField(max_length=100, unique=True)
        
    def __str__(self):
        return self.nombre
    
    
class Opcion(models.Model):
    nombre = models.CharField(max_length=100, unique=True)
    partido = models.ForeignKey('Partido', null=True)    # blanco / nulo / etc. 
    
    def __str__(self):
        return self.nombre
    
class VotoMesa(models.Model):
    mesa = models.ForeignKey('Mesa', related_name='resultados')
    categoria = models.CharField(max_length=20, choices=(('dip', 'Diputados'), ('sen', 'Senadores')))
    opcion = models.ForeignKey('Opcion')
    votos = models.IntegerField()
    
    def __str__(self):
        return u"%s: %s -> %d" % (self.get_categoria_display(), self.opcion, self.votos)
    
    
    class Meta:
        unique_together = ('mesa', 'opcion', 'categoria')

## Importación de datos a los modelos


In [3]:
from csv import DictReader

In [4]:
Seccion.objects.all()

<QuerySet []>

Importo secciones, circuitos y mesas de los distintos archivos

In [5]:
for seccion in DictReader(open('secciones.csv')):
    Seccion.objects.create(**seccion)
    

In [6]:
secciones = {s.numero: s for s in Seccion.objects.all()}
circuitos = []
for circuito in DictReader(open('circuitos.csv')):
    circuito['seccion'] = secciones[circuito['seccion']] 
    circuitos.append(Circuito(**circuito))
Circuito.objects.bulk_create(circuitos);

In [7]:
circuitos = {c.numero: c for c in Circuito.objects.all()}
mesas = []
for mesa in DictReader(open('mesas.csv')):
    mesa['circuito'] = circuitos[mesa['circuito']]
    mesas.append(Mesa(**mesa))
Mesa.objects.bulk_create(mesas);

El archivo `resultados.csv` pesa casi 100mb y tiene una fila por cada mesa-opcion, para senadores y diputados. 
Para cargarlo más rapido (sino puede tardar muchisimo) hice unas pequeñas optimizaciones. 

- Importo partidos y listas que existan primero y armo un diccionario para no tener que hacer consultas luego
- Hago "chunks" e inserto varias filas de resultados a la vez. 

Con esto tarda unos 5' en cargar. 

In [8]:
from itertools import zip_longest

def grouper(n, iterable, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

In [9]:
import pandas as pd
partidos_listas = pd.read_csv('resultados.csv', usecols=['partido','lista'])
partidos_listas.drop_duplicates(inplace=True)


In [10]:
partidos_listas.fillna('', inplace=True)
partidos_listas

,lista,partido
0,,nulos
1,,blancos
2,,recurridos
3,,impugnados
4,NARANJA,HUMANISTA
5,MARGARITA,GEN
6,CORDOBA CAMBIA,PRIMERO LA GENTE
7,VALORES EN ACCION,ENCUENTRO VECINAL CORDOBA
8,NUEVO PAIS,POLITICA ABIERTA PARA LA INTEGRIDAD SOCIAL
9,POR UN NUEVO PAIS,POLITICA ABIERTA PARA LA INTEGRIDAD SOCIAL


In [11]:
for _, row in partidos_listas.iterrows():
    if not row.lista:
        Opcion.objects.create(nombre=row.partido)
    else:
        partido, _ = Partido.objects.get_or_create(nombre=row.partido)
        Opcion.objects.create(nombre=row.lista, partido=partido)

In [12]:
opciones = {o.nombre: o for o in Opcion.objects.all()}
opciones

{'CAMBIANDO JUNTOS': <Opcion: CAMBIANDO JUNTOS>,
 'COMPROMISO': <Opcion: COMPROMISO>,
 'CORDOBA CAMBIA': <Opcion: CORDOBA CAMBIA>,
 'JUNTOS POR PAIS': <Opcion: JUNTOS POR PAIS>,
 'MARGARITA': <Opcion: MARGARITA>,
 'MOVIMIENTO DE UNIDAD PERONISTA': <Opcion: MOVIMIENTO DE UNIDAD PERONISTA>,
 'NARANJA': <Opcion: NARANJA>,
 'NUEVO PAIS': <Opcion: NUEVO PAIS>,
 'PARA LA DEMOCRACIA SOCIAL': <Opcion: PARA LA DEMOCRACIA SOCIAL>,
 'POR UN NUEVO PAIS': <Opcion: POR UN NUEVO PAIS>,
 'POR UN PAIS MEJOR': <Opcion: POR UN PAIS MEJOR>,
 'ROJA': <Opcion: ROJA>,
 'UNIDAD': <Opcion: UNIDAD>,
 'UNIDAD DE LA  IZQUIERDA': <Opcion: UNIDAD DE LA  IZQUIERDA>,
 'VALORES EN ACCION': <Opcion: VALORES EN ACCION>,
 'blancos': <Opcion: blancos>,
 'impugnados': <Opcion: impugnados>,
 'nulos': <Opcion: nulos>,
 'recurridos': <Opcion: recurridos>}

In [19]:
mesas = {m.fake_id: m for m in Mesa.objects.all()}
VotoMesa.objects.all().delete()

(45528, {'orm_magic.VotoMesa': 45528})

In [21]:
for chunk in grouper(300, DictReader(open('resultados.csv'))):
    resultados = []
    for resultado in chunk:
        if not resultado:
            continue
        mesa = mesas[resultado['mesa']]
        lista = resultado['lista'] or resultado['partido']
        opcion = opciones[lista]
        try:
            votos = int(resultado['votos'] or 0)
        except ValueError:
            print('**votos no validos', resultado)
            continue
        
        resultados.append(VotoMesa(mesa=mesa, 
                               opcion=opcion, 
                               votos=votos, 
                               categoria=resultado['categoria']))
    VotoMesa.objects.bulk_create(resultados)

<div id="resultados"></div>

## RESULTADOS 



Teniendo la base de datos puedo hacer cualquier tipo de consulta, por mesa, por circuito, totales, etc. 

Primero calculemos los porcentajes totales para diputados y senadores

In [58]:
for o in Opcion.objects.all():
    print(o.id, o.nombre, o.partido)

1 nulos None
2 blancos None
3 recurridos None
4 impugnados None
5 NARANJA HUMANISTA
6 MARGARITA GEN
7 CORDOBA CAMBIA PRIMERO LA GENTE
8 VALORES EN ACCION ENCUENTRO VECINAL CORDOBA
9 NUEVO PAIS POLITICA ABIERTA PARA LA INTEGRIDAD SOCIAL
10 POR UN NUEVO PAIS POLITICA ABIERTA PARA LA INTEGRIDAD SOCIAL
11 POR UN PAIS MEJOR POLITICA ABIERTA PARA LA INTEGRIDAD SOCIAL
12 JUNTOS POR PAIS POLITICA ABIERTA PARA LA INTEGRIDAD SOCIAL
13 ROJA FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES
14 COMPROMISO SOMOS
15 MOVIMIENTO DE UNIDAD PERONISTA UNION POR CORDOBA
16 UNIDAD FRENTE CORDOBA CIUDADANA
17 UNIDAD DE LA  IZQUIERDA IZQUIERDA AL FRENTE POR EL SOCIALISMO
18 CAMBIANDO JUNTOS CAMBIEMOS
19 PARA LA DEMOCRACIA SOCIAL CAMBIEMOS


In [32]:
UPC = [15]
CAMBIEMOS = [18, 19]
FCC = [16]
FIT = [13]
TODOS = {'UPC': UPC, 'CAMBIEMOS': CAMBIEMOS, 'FCC': FCC, 'FIT': FIT}

In [28]:
total_positivos = VotoMesa.objects.filter(opcion__partido__isnull=False).aggregate(v=Sum('votos'))['v']
total_positivos

1874318

In [29]:
from django.db.models import Avg, Sum

In [33]:
results = {}
for part, ids in TODOS.items():
    vot = VotoMesa.objects.filter(opcion__id__in=ids).aggregate(v=Sum('votos'))['v']
    results[part] =  vot, vot/total_positivos
    

In [34]:
results

{'CAMBIEMOS': (850778, 0.45391337008981403),
 'FCC': (189087, 0.10088309454425556),
 'FIT': (82584, 0.044060826391252715),
 'UPC': (546197, 0.2914110625838305)}

### Mesas donde el FCC sacó menos del 30% de los votos promedio en su circuito

In [39]:
from IPython.display import display_html, HTML

In [43]:
mesas = []
for circuito in Circuito.objects.all():
    promedio_circuito = VotoMesa.objects.filter(mesa__circuito=circuito, opcion__id__in=FCC).aggregate(p=Avg('votos'))['p']
    if not promedio_circuito:
        continue
    
    for mesa in Mesa.objects.filter(circuito=circuito, resultados__votos__gt=0, 
                                    resultados__votos__lte=int(promedio_circuito * .3), 
                                    resultados__opcion__id__in=FCC):
        vot = mesa.resultados.get(opcion__id__in=FCC).votos
        mesas.append((mesa, vot, promedio_circuito))


In [44]:
len(mesas)

34

In [45]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Votos FCC</th><th>Promedio FCC/Circ</th></tr>'
for l in mesas:
    mesa, vot, promedio_circuito = l
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados} </td><td>{vot}</td><td> {promedio_circuito:.2f}</td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa,Circuito,Computados,Votos FCC,Promedio FCC/Circ
Mesa 04998,Circuito 0126 (009 - Marcos Juárez),238,3,11.11
Mesa 05103,Circuito 0133 (009 - Marcos Juárez),219,4,18.76
Mesa 05520,Circuito 0156 (012 - Punilla),189,4,22.79
Mesa 04246,Circuito 0060 (005 - General Roca),237,3,17.60
Mesa 04100,Circuito 0044 (004 - Cruz del Eje),194,3,29.52
Mesa 06413,Circuito 0213 (014 - Río Primero),198,1,3.75
Mesa 03434,Circuito 0028 (003 - Colón),229,5,17.27
Mesa 08376,Circuito 0373 (026 - Unión),240,11,40.75
Mesa 05933,Circuito 0179 (013 - Río Cuarto),244,1,25.57
Mesa 04450,Circuito 0089 (006 - General San Martín),255,1,35.54


### Mesas no computadas

In [46]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th></tr>'
for mesa in Mesa.objects.filter(resultados__votos=0).distinct():
    if mesa.resultados.all().aggregate(p=Avg('votos'))['p'] != 0:
        # no computadas
        continue        
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa,Circuito
Mesa 03903,Circuito 0040 (003 - Colón)
Mesa 03897,Circuito 0040 (003 - Colón)
Mesa 03938,Circuito 0040A (003 - Colón)
Mesa 03872,Circuito 0040 (003 - Colón)
Mesa 04714,Circuito 0095 (007 - Ischilin)
Mesa 06496,Circuito 0227D (015 - Río Seco)
Mesa 04043,Circuito 0041A (003 - Colón)
Mesa 03254,Circuito 0014Q (001 - Capital)
Mesa 03253,Circuito 0014Q (001 - Capital)
Mesa 02910,Circuito 0014 (001 - Capital)


### Mesas para los que  FCC sacó 0 votos

In [48]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Promedio UC/Circ</th></tr>'
for mesa in Mesa.objects.filter(resultados__votos=0, resultados__opcion__id__in=FCC).distinct():
    if mesa.resultados.all().aggregate(p=Avg('votos'))['p'] == 0:
        # ignorar no computadas
        continue
    
    promedio_circuito = VotoMesa.objects.filter(mesa__circuito=mesa.circuito, opcion__id__in=FCC).aggregate(p=Avg('votos'))['p']
    
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados} </td><td> {promedio_circuito:.2f} </td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa,Circuito,Computados,Promedio UC/Circ
Mesa 07304,Circuito 0290 (020 - San Justo),227,7.00
Mesa 03861,Circuito 0039 (003 - Colón),30,0.00
Mesa 06498,Circuito 0228 (015 - Río Seco),168,16.88
Mesa 04681,Circuito 0095 (007 - Ischilin),186,30.69
Mesa 02713,Circuito 0013H (001 - Capital),193,19.54
Mesa 02703,Circuito 0013H (001 - Capital),213,19.54
Mesa 02759,Circuito 0013I (001 - Capital),227,26.00
Mesa 01961,Circuito 0011G (001 - Capital),230,20.29
Mesa 02376,Circuito 0012F (001 - Capital),225,23.57
Mesa 01748,Circuito 0011A (001 - Capital),8,24.62


### Mesas para los que Cambiemos sacó 190% (o más) del promedio de su circuito

In [63]:
mesas = []
for circuito in Circuito.objects.all():
    promedio_circuito = VotoMesa.objects.filter(mesa__circuito=circuito, opcion__id__in=CAMBIEMOS).aggregate(p=Avg('votos'))['p']
    if not promedio_circuito:
        continue
    
    for mesa in Mesa.objects.filter(circuito=circuito, resultados__votos__gt=0, 
                                    resultados__votos__gte=promedio_circuito*1.9, 
                                    resultados__opcion__id__in=CAMBIEMOS):
        vot = mesa.resultados.filter(opcion__id__in=CAMBIEMOS).aggregate(p=Sum('votos'))['p']
        mesas.append((mesa, vot, promedio_circuito))


In [62]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Votos CAMBIEMOS</th><th>Promedio CAMBIEMOS/Circ</th></tr>'
for l in mesas:
    mesa, vot, promedio_circuito = l
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados} </td><td>{vot}</td><td> {promedio_circuito:.2f}</td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa,Circuito,Computados,Votos CAMBIEMOS,Promedio CAMBIEMOS/Circ
Mesa 06478,Circuito 0223 (015 - Río Seco),117,70,31.33
Mesa 06482,Circuito 0225 (015 - Río Seco),171,47,12.67
Mesa 06493,Circuito 0227B (015 - Río Seco),132,56,26.75
Mesa 06497,Circuito 0227D (015 - Río Seco),143,78,19.50
Mesa 06505,Circuito 0228 (015 - Río Seco),208,83,37.38
Mesa 06504,Circuito 0228 (015 - Río Seco),188,96,37.38
Mesa 06500,Circuito 0228 (015 - Río Seco),189,87,37.38
Mesa 06514,Circuito 0229 (015 - Río Seco),212,95,41.20
Mesa 06509,Circuito 0229 (015 - Río Seco),234,104,41.20
Mesa 06508,Circuito 0229 (015 - Río Seco),234,112,41.20


### Mesas con muchos votos nulos

In [66]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Votos Nulos</th></tr>'
for mesa in Mesa.objects.filter(resultados__opcion__id=1, resultados__votos__gte=18).distinct():
    vot = mesa.resultados.get(opcion__id=1).votos
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados} </td><td>{vot}</td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa,Circuito,Computados,Votos Nulos
Mesa 03869,Circuito 0040 (003 - Colón),257,19
Mesa 03866,Circuito 0040 (003 - Colón),246,19
Mesa 04685,Circuito 0095 (007 - Ischilin),239,25
Mesa 02948,Circuito 0014A (001 - Capital),240,18
Mesa 03099,Circuito 0014H (001 - Capital),247,19
Mesa 03209,Circuito 0014N (001 - Capital),239,21
Mesa 02615,Circuito 0013D (001 - Capital),253,22
Mesa 02683,Circuito 0013G (001 - Capital),237,18
Mesa 02730,Circuito 0013H (001 - Capital),171,18
Mesa 02646,Circuito 0013G (001 - Capital),259,21
